& is written as &amp ;
@gautam can be replaced with 'username'

\# can be considered as a seperate token

remove empty strings that may app

String that contains http can be replaced with URL
Special chars
-
()


In [20]:
def mySplit(string):
    string = string[1:-1]
    res = []
    In = True
    delim = ["'",'"']
    prev = string[0]
    word = ''
    i = 1
    while(i<len(string)):
        if((not In) and (string[i] in delim)):
            In = True
            prev = string[i]
            i+=1
        elif(i == len(string)-2):
            res.append(word+string[i])
            word = ''
            i+=2
        elif(In and (string[i:i+3] == prev+', ')):
            res.append(word)
            In = False
            prev = ''
            word = ''
            i+=3
        else:
            word+=string[i]
            i+=1
    return res

In [21]:
sentence = '''['""who', ' may', ' (or', ' may', ' not', ') have', ' it', '"" - Schrodinger\'s', ' Virus', '.  We', ' can', ' not', ' get', ' tested', ', so', ' we', ' have', ' to', ' act', ' like', ' we', ' have', ' the', ' virus', ' so', ' we', ' do', ' not', ' spread', ' it', '.  We', ' also', ' have', ' to', ' act', ' like', ' we', ' have', ' never', ' had', ' the', ' virus', ', because', ' if', ' we', ' have', ' not', ' had', ' the', ' virus', ', we', ' are', ' not', ' immune', '.#COVID19']'''
slist = mySplit(sentence)
print(slist, len(slist))

['""who', ' may', ' (or', ' may', ' not', ') have', ' it', '"" - Schrodinger\'s', ' Virus', '.  We', ' can', ' not', ' get', ' tested', ', so', ' we', ' have', ' to', ' act', ' like', ' we', ' have', ' the', ' virus', ' so', ' we', ' do', ' not', ' spread', ' it', '.  We', ' also', ' have', ' to', ' act', ' like', ' we', ' have', ' never', ' had', ' the', ' virus', ', because', ' if', ' we', ' have', ' not', ' had', ' the', ' virus', ', we', ' are', ' not', ' immune', '.#COVID19'] 55


In [25]:
def getTag(i, spans, spane):
    if i in spans:
        return 'B'
    for j in range(len(spans)):
        if(spans[j]<i<=spane[j]):
            return 'I'
    return 'O'

In [33]:
import pandas as pd
import re
import typing


def formatWord(word: str):
    '''INPUT: string
    OUTPUT: list of tokens. Assumption - the tokens are in reduced form, easy to understand for machine
    '''
    word = word.replace('(',' ( ')
    word = word.replace('\\n','\n')
    word = word.replace('/', ' / ' )
    try:
        word = re.sub("\.[\.]+", " ... ", word)
        word = re.sub('"', ' " ', word)
    except Exception as e:
        print(word)
        raise e
    words = word.split()
    res = []
    for i in range(len(words)):
        if(words[i][0] == '#'):
            res.append('#')
            res.append(words[i][1:])
        elif(words[i][0] == '@'):
            res.append('username')
        elif('http' in word):
            res.append('URL')
            break
        elif('&amp;'==words[i]):
            res.append('&')
        elif(words[i] == ''):
            continue
        elif('-' in words[i]):
            tks = words[i].split('-')
            for j in range(len(tks)-1):
                res.append(tks[j])
                res.append('-')
            res.append(tks[-1])
        elif('.' in words[i]):
            tks = words[i].split('.')
            for j in range(len(tks)-1):
                res.append(tks[j])
                res.append('.')
            res.append(tks[-1])
        else:
            res.append(words[i])
    del_list = []
    for i in range(len(res)):
        if(res[i] == ''):
            del_list.append(i)
    del_list.sort(reverse=True)
    for d in del_list:
        del res[d]
    return res

def formatSlist(slist):
    '''
    INPUT: Words list seperated by ', '
    OUTPUT: List of list of pairs, pair[0] = word id for reverse mapping the results, pair[1] = reduced token
    '''
    new_list = []
    for i in range(len(slist)):
        words = formatWord(slist[i])
        for word in words:
            new_list.append((i, word))
    return new_list
        
    

df = pd.read_csv('Data/train.csv')
pattern = "','"

new_df = {'sentence_id':[], 'word_id':[],'word': [], 'tag':[]}
sentence_id = 0
for i in range(len(df)):
    sentence = df.iloc[i,0]
    if(int(df.iloc[i,1]) != 1):
        print(i)
    if(df.iloc[i,2] == '[]'):
        spans = []
        spane = []
    else:
        spans = list(map(int, str(df.iloc[i,2])[1:-1].split(',')  ))
        spane = list(map(int,str(df.iloc[i,3])[1:-1].split(',') ))
    if(len(spans)!=len(spans)):
        print(spans)
        print(spane)
        break
    slist = mySplit(sentence)
    
    slist = formatSlist(slist)
    

    for j in range(len(slist)):
        new_df['sentence_id'].append(sentence_id)
        new_df['word_id'].append(slist[j][0])
        new_df['word'].append(slist[j][1])
        new_df['tag'].append(getTag(slist[j][0], spans, spane))
    sentence_id+=1
new_df = pd.DataFrame(new_df)
#new_df.to_csv('Formatted.csv')

In [34]:
new_df.to_csv('Formatted.csv')